In [1]:
import pandas as pd
import requests
import wget

import folium
from folium.plugins import MarkerCluster

In [2]:
df0 = pd.read_excel("uni_addresses.xlsx")
df0.head()

,name,address,population
0,Abertay University,"Bell St, Dundee DD1 1HG",3665
1,Edinburgh Napier University,"9 Sighthill Ct, Edinburgh EH11 4BN",9365
2,Glasgow Caledonian University,"Cowcaddens Rd, Glasgow G4 0BA",12185
3,Glasgow School of Art,"167 Renfrew St, Glasgow G3 6RQ",1605
4,Heriot-Watt University,"Campus The Avenue, Edinburgh EH14 4AS",7605


In [3]:
def query_address(address):
    """Return response from open streetmap.
    
    Parameter:
    address - address of establishment
    
    Returns:
    result - json, response from open street map
    """
    
    url = "https://nominatim.openstreetmap.org/search"
    parameters = {'q':'{}, Scotland'.format(address), 'format':'json'}
    
    response = requests.get(url, params=parameters)
    # don't want to raise an error to not stop the processing
    # print address instead for future inspection
    if response.status_code != 200:
        print("Error querying {}".format(address))
        result = {}
    else:
        result = response.json()
    return result

In [4]:
df0['json'] = df0['address'].map(lambda x: query_address(x))

# drop empty responses
df1 = df0[df0['json'].map(lambda d: len(d)) > 0].copy()
print(df1.shape[0])

17


In [5]:
# extract relevant fields from API response (json format)
df1['lat'] = df1['json'].map(lambda x: x[0]['lat'])
df1['lon'] = df1['json'].map(lambda x: x[0]['lon'])
df1['type'] = df1['json'].map(lambda x: x[0]['type'])

In [6]:
df1

,name,address,population,json,lat,lon,type
0,Abertay University,"Bell St, Dundee DD1 1HG",3665,"[{'place_id': 188831393, 'licence': 'Data © Op...",56.4634314,-2.9725875,unclassified
1,Edinburgh Napier University,"9 Sighthill Ct, Edinburgh EH11 4BN",9365,"[{'place_id': 147312009, 'licence': 'Data © Op...",55.9235783,-3.2870856,unclassified
2,Glasgow Caledonian University,"Cowcaddens Rd, Glasgow G4 0BA",12185,"[{'place_id': 179549423, 'licence': 'Data © Op...",55.8657779,-4.2499464,tertiary
3,Glasgow School of Art,"167 Renfrew St, Glasgow G3 6RQ",1605,"[{'place_id': 104781278, 'licence': 'Data © Op...",55.86605315,-4.263740052313956,attraction
4,Heriot-Watt University,"Campus The Avenue, Edinburgh EH14 4AS",7605,"[{'place_id': 85200278, 'licence': 'Data © Ope...",55.91029585,-3.323447514558347,university
5,"Queen Margaret University, Edinburgh","Queen Margaret University Way, Musselburgh EH2...",3105,"[{'place_id': 98879825, 'licence': 'Data © Ope...",55.9222238,-3.0667181,residential
6,Robert Gordon University,"Garthdee House, Garthdee Rd, Garthdee, Aberdee...",8080,"[{'place_id': 160480173, 'licence': 'Data © Op...",57.11834245,-2.145071748221806,university
7,Royal Conservatoire of Scotland,"100 Renfrew St, Glasgow G2 3DB",900,"[{'place_id': 220452687, 'licence': 'Data © Op...",55.86638255,-4.257549087975787,university
9,The University of Aberdeen,"King's College, Aberdeen AB24 3FX",9510,"[{'place_id': 257921001, 'licence': 'Data © Op...",57.1639306,-2.100875779418921,university
10,The University of Dundee,"Nethergate, Dundee DD1 4HN",10255,"[{'place_id': 95979298, 'licence': 'Data © Ope...",56.4573119,-2.9758871,tertiary


In [7]:
df1.sort_values(by="population", ascending=False, inplace=True)

In [8]:
# Scotland coordinates
scotland_coordinates = (56.4907, -4.2026)

# create empty map zoomed in on Scotland
map = folium.Map(width=600,height=600,location=scotland_coordinates, zoom_start=7)

# add one markercluster per type to allow for individual toggling
uni = MarkerCluster(name='university').add_to(map)

# add universities to the map
for population in df1.iterrows():
    folium.Marker(location=[float(population[1]['lat']), float(population[1]['lon'])], 
                  popup=population[1]['name'],
                 icon=folium.Icon(color='green', icon_color='white', icon='university', angle=0, prefix='fa'))\
    .add_to(uni)
    
# enable toggling of data points
folium.LayerControl().add_to(map)    
    
display(map)

In [9]:
# Scotland coordinates
scotland_coordinates = (56.4907, -4.2026)

# create empty map zoomed in on Scotland
m = folium.Map(width=600,height=600,location=scotland_coordinates, zoom_start=7)

# add marker one by one on the map
for i in range(0,len(df1)):
   folium.Circle(
      location=[df1.iloc[i]['lat'], df1.iloc[i]['lon']],
      popup=df1.iloc[i]['name'],
      radius=float(df1.iloc[i]['population'])*1,
      color='green',
      fill=True,
      fill_color='green'
   ).add_to(m)

display(m)

In [10]:
# Scotland coordinates
scotland_coordinates = (56.4907, -4.2026)

# create empty map zoomed in on Scotland
n = folium.Map(width=600,height=600,location=scotland_coordinates, zoom_start=7)

# add marker one by one on the map
for i in range(0,len(df1)):
   folium.CircleMarker(
      location=[df1.iloc[i]['lat'], df1.iloc[i]['lon']],
      popup=df1.iloc[i]['name'],
      radius=float(df1.iloc[i]['population'])/700,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(n)

display(n)